# PokeClassifier

Gen1 Pokemon Image Classifier with Automatic Image Standardization

## Imports and Data Loading

In [2]:
# Import
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### Model Hyperparameters

In [3]:
batch_size = 32
image_shape = (256,256,3)

In [4]:
train_path = 'TrainImages'
test_path = 'TestImages'


image_gen = ImageDataGenerator(rotation_range=20,
                               rescale = 1./255,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')

train_image_gen = image_gen.flow_from_directory(train_path,
                                                target_size=image_shape[:2],
                                                color_mode='rgb',
                                                batch_size=batch_size,
                                                class_mode='categorical')

test_image_gen = image_gen.flow_from_directory(test_path,
                                                target_size=image_shape[:2],
                                                color_mode='rgb',
                                                batch_size=batch_size,
                                                class_mode='categorical')

Found 5830 images belonging to 149 classes.
Found 949 images belonging to 149 classes.


### Model Definition

In [5]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
model = Sequential()

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(BatchNormalization())
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())

model.add(Dense(1024,activation='relu'))

model.add(Dropout(rate=0.25))

model.add(Dense(512,activation='relu'))

model.add(Dense(149,activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer=Adagrad(learning_rate=0.001), metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 128)      0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 64, 64, 128)       0         
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 128)      5

### Training

In [7]:
early_stopping = EarlyStopping(monitor='val_loss',patience=3)
results = model.fit(train_image_gen,epochs=80,validation_data=test_image_gen,callbacks=[early_stopping])

Epoch 1/80


KeyboardInterrupt: 

### Evalute Model

In [ ]:
loss_history = pd.DataFrame(model.history.history)

model.evaluate(test_image_gen)
loss_history[['loss','val_loss']].plot()
loss_history[['accuracy','val_accuracy']].plot()